In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json
from jsonschema import validate, ValidationError

# Załaduj schemat z pliku
with open('sensor_schema.json') as f:
    schema = json.load(f)

consumer = KafkaConsumer(
    'lab5-topic',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='lab5-validator',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

producer = KafkaProducer(
    bootstrap_servers='kafka_streaming_lab:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

print("Validator started...")

with open("invalid_messages.log", "w") as logfile:
    for msg in consumer:
        data = msg.value
        try:
            validate(instance=data, schema=schema)
            producer.send('lab5-topic-valid', value=data)
            print(f"Valid message: {data}")
        except ValidationError as e:
            print(f"Invalid message: {data}")
            logfile.write(f"{json.dumps(data)}\n")
